In [73]:
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import requests
from unicodedata import normalize
import re, copy, random, time
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar
from uberPrice import getPriceNow

ModuleNotFoundError: No module named 'uber_rides'

In [57]:
#Step 1, Read the dataset with features, clean any unecessary stuff
cab_data = pd.read_csv("data_with_features.csv")
del cab_data["Unnamed: 0"]
del cab_data["Unnamed: 0.1"]
del cab_data["Unnamed: 0.1.1"]
#Print columns to validate
print(cab_data.columns.values)

['dropoff_latitude' 'dropoff_longitude' 'pickup_latitude'
 'pickup_longitude' 'total_amount' 'tpep_dropoff_datetime'
 'tpep_pickup_datetime' 'start_latlng' 'end_latlng' 'distance'
 'tpep_dropoff_datetime_years' 'tpep_dropoff_datetime_months'
 'tpep_dropoff_datetime_days' 'tpep_dropoff_datetime_hours'
 'tpep_dropoff_datetime_minutes' 'tpep_dropoff_datetime_seconds'
 'tpep_dropoff_datetime_day_of_week' 'tpep_dropoff_datetime_is_holiday'
 'tpep_pickup_datetime_years' 'tpep_pickup_datetime_months'
 'tpep_pickup_datetime_days' 'tpep_pickup_datetime_hours'
 'tpep_pickup_datetime_minutes' 'tpep_pickup_datetime_seconds'
 'tpep_pickup_datetime_day_of_week' 'tpep_pickup_datetime_is_holiday'
 'duration' 'Mean Temperature' 'Max Temperature' 'Min Temperature'
 'Dew Point' 'Average Humidity' 'Precipitation' 'Snow' 'Wind Speed'
 'Visibility']


In [59]:
#Filter out any nonzero price values
price_filter = cab_data["total_amount"] > 0
cab_data = cab_data[price_filter]

print("Data size: ", len(cab_data))

#Filter out any negative distance values
dist_filter = cab_data['distance'] > 0
cab_data = cab_data[dist_filter]

data_size = len(cab_data)
print("Data size: ", data_size)

Data size:  978979
Data size:  978979


Okay great! The data is clean and is ready to be processed!

In [65]:
#Step 1: Determine interval for sampling Uber API
time_period = 24*60 #In minutes
sample_rate = 5 #In 1 sample per *sample_rate* minutes
num_samples = time_period // sample_rate
print("We will be making %d calls to uber API." % num_samples)

#Step 2: Get random subset of data to use as test points
sample_data = cab_data.sample(n=num_samples)
sample_data = sample_data[['start_latlng', 'end_latlng', 'distance', 'duration']]
#print(sample_data)

We will be making 288 calls to uber API.


In [61]:
#Step 3: Initialize weather and datetime features for our sample data
start_date = datetime.now()
end_date = datetime.now()
print(start_date, end_date)
date_features = init_weather_features(start_date, end_date)

2018-05-05 13:46:59.683784 2018-05-05 13:46:59.683784
2018 5 5


In [71]:
for (idx,row) in sample_data.iterrows():
    curr_time = datetime.now()
    
    
    
    time.sleep(5)

start_latlng    40.788288, -73.978569
end_latlng      40.793098, -73.967430
distance                      0.67188
duration                            9
Name: 668600, dtype: object
start_latlng    40.728634, -73.976318
end_latlng      40.723362, -73.988319
distance                     0.727481
duration                            7
Name: 607382, dtype: object
start_latlng    40.725578, -73.999916
end_latlng      40.735634, -74.008118
distance                     0.816604
duration                            7
Name: 683101, dtype: object


KeyboardInterrupt: 

In [45]:
feature_names = ["distance", 'tpep_dropoff_datetime_years', 'tpep_dropoff_datetime_months',
       'tpep_dropoff_datetime_days', 'tpep_dropoff_datetime_hours',
       'tpep_dropoff_datetime_minutes', 'tpep_dropoff_datetime_seconds',
       'tpep_dropoff_datetime_day_of_week',
       'tpep_dropoff_datetime_is_holiday', 'tpep_pickup_datetime_years',
       'tpep_pickup_datetime_months', 'tpep_pickup_datetime_days',
       'tpep_pickup_datetime_hours', 'tpep_pickup_datetime_minutes',
       'tpep_pickup_datetime_seconds', 'tpep_pickup_datetime_day_of_week',
       'tpep_pickup_datetime_is_holiday', 'Mean Temperature',
       'Max Temperature', 'Min Temperature', 'Dew Point',
       'Average Humidity', 'Precipitation', 'Snow', 'Wind Speed',
       'Visibility']

print(len(feature_names))

26
